# Importing and Transforming raw DATA

In [1]:
import numpy as np
import pandas as pd
import os
from nltk.tokenize import RegexpTokenizer
from nltk.stem import *

folder = os.getcwd() ; print folder

/home/arda/Documents/youtube


In [164]:
pd_train = pd.read_csv(folder + '/data/train_sample.csv', sep=',', dialect=None, compression='infer', doublequote=True, 
            escapechar='\\', quotechar='"', quoting=0, skipinitialspace=False,
            lineterminator=None, header='infer', index_col=None, names=None,
            prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None,
            na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None,
            dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False,
            na_filter=True, compact_ints=False, use_unsigned=False, low_memory=False, buffer_lines=None,
            warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None,
            decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None,
            memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None,
            verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False,
            infer_datetime_format=False, skip_blank_lines=True)


pd_test = pd.read_csv(folder + '/data/test_sample.csv', sep=',', dialect=None, compression='infer', doublequote=True, 
            escapechar='\\', quotechar='"', quoting=0, skipinitialspace=False,
            lineterminator=None, header='infer', index_col=None, names=None,
            prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None,
            na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None,
            dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False,
            na_filter=True, compact_ints=False, use_unsigned=False, low_memory=False, buffer_lines=None,
            warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None,
            decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None,
            memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None,
            verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False,
            infer_datetime_format=False, skip_blank_lines=True)

In [165]:
pd_train.shape

(239225, 16)

In [166]:
print pd_train.columns

Index([u'video_category_id', u'title', u'description', u'published_at',
       u'viewCount', u'likeCount', u'dislikeCount', u'favoriteCount',
       u'commentCount', u'duration', u'dimension', u'definition', u'caption',
       u'licensedContent', u'topicIds', u'relevantTopicIds'],
      dtype='object')


In [167]:
#split topicIds
pd_train[u'topicIds'] = pd_train[u'topicIds'].apply(lambda r: r.split(";") if type(r)==str else [])
pd_test[u'topicIds'] = pd_test[u'topicIds'].apply(lambda r: r.split(";") if type(r)==str else [])

#split
pd_train[u'relevantTopicIds'] = pd_train[u'relevantTopicIds'].apply(lambda r: r.split(";") if type(r)==str else [])
pd_test[u'relevantTopicIds'] = pd_test[u'relevantTopicIds'].apply(lambda r: r.split(";") if type(r)==str else [])

In [168]:
import datetime
from dateutil import parser
pd_train['published_at'] =  pd_train['published_at'].apply(lambda r: parser.parse(r[:19]) if len(r)==24 else datetime.datetime.now())
pd_test['published_at'] =  pd_test['published_at'].apply(lambda r: parser.parse(r[:19]) if len(r)==24 else datetime.datetime.now())

In [192]:
pd_train['dimension'].unique()

array(['2d', '3d', 'hd', nan], dtype=object)

In [193]:
print pd_train['definition'].unique()
nb_2D = pd_train['dimension'].apply(lambda r: type(r)!=str)

['hd' 'sd' 'false' nan]


,video_category_id,title,description,published_at,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,duration,dimension,definition,caption,licensedContent,topicIds,relevantTopicIds
23580,23,500K SUBSCRIBER GIVEAWAY! [MacBook Pro + iPad ...,"""500K SUBSCRIBER GIVEAWAY! [MacBook Pro + IPad...",2014-08-12 19:52:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
27176,23,How To Make Your Lips BIGGER In 3 Minutes!,"""How To Make Your Lips BIGGER In 3 Minutes!"" G...",2014-10-24 22:00:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


array(['hd', 'sd', 'false', nan], dtype=object)

In [ ]:
df = df[np.isfinite(df['EPS'])]